# Выявление особенностей словоупотребления в текстах Л. Н. Толстого методами дистрибутивной семантики

** TO DO **

- обучить сегментатор поточнее
- проверить его на "медленной" и "быстрой" версиях (по количеству предложений)
- проверить теги для дневников и писем
- почистить мета-информацию из дневников и писем
- "скормить" майстему дореволюционный (вроде ест)

In [52]:
import os
import re
from lxml import etree
from pymystem3 import Mystem
import string
import nltk
#from langdetect import detect
import langid
from datetime import datetime
mystem = Mystem()

## Сегментатор

In [2]:
language_punkt_vars = nltk.tokenize.punkt.PunktLanguageVars
language_punkt_vars.sent_end_chars=('.','?','!','»','…')
with open('.\\data\\traindata.txt', 'r', encoding='utf-8') as f:
    traindata = f.read()
trainer = nltk.tokenize.punkt.PunktTrainer(traindata, language_punkt_vars)
trainer.INCLUDE_ALL_COLLOCS = True 
trainer.INCLUDE_ABBREV_COLLOCS = True
params = trainer.get_params()
sbd = nltk.tokenize.punkt.PunktSentenceTokenizer(params)

  Abbreviation: [4.0363] н
  Abbreviation: [1.4849] ст
  Abbreviation: [5.4625] тыс
  Abbreviation: [0.6029] матф
  Abbreviation: [4.0363] д
  Abbreviation: [1.0925] руб
  Abbreviation: [19.3033] см
  Abbreviation: [0.5463] стр
  Abbreviation: [8.0726] м
  Abbreviation: [4.0363] г
  Abbreviation: [1.4849] кн
  Abbreviation: [5.9395] т.п
  Abbreviation: [4.0363] п
  Abbreviation: [1.4849] ул
0.000380517503805175 0.007662835249042145 0.0 5256 261 2 2
0.000380517503805175 0.007662835249042145 0.0 5256 261 2 2
0.0030441400304414 0.007662835249042145 0.0028028028028028026 5256 261 16 2
0.009512937595129375 0.038314176245210725 0.008008008008008008 5256 261 50 10
0.0030441400304414 0.007662835249042145 0.0028028028028028026 5256 261 16 2
0.001141552511415525 0.007662835249042145 0.0008008008008008008 5256 261 6 2
0.0036149162861491627 0.007662835249042145 0.0034034034034034033 5256 261 19 2
0.00076103500761035 0.007662835249042145 0.0004004004004004004 5256 261 4 2
0.0015220700152207 0.02298

## Данные

На основе 90-томного собрания сочинений

In [3]:
tei_ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

In [4]:
string.punctuation = string.punctuation.replace('-','')
string.punctuation

'!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

## Препроцессинг

In [5]:
def get_one_writing(filename):
    root = etree.parse(filename)#.getroot()
    for i in tags_for_deletion:
        for j in root.findall(i, tei_ns):
            parent = j.find("..")
            parent.remove(j)
    for i in tags_to_strip:
        etree.strip_tags(root, i)
    return root.find(".//tei:text", tei_ns).text.strip()

In [46]:
def preprocessing(text, f):
    #text = re.sub('[0-9—A-Za-zÉéÂâÊêÎîÔôÛûÀàÈèÙùËëÏïÜüŸÿÇç«»’§°¼½×áä˃<]+','', text)
    text = re.sub('[\n\s]+',' ', text)
    lng = len(sbd.sentences_from_text(text, realign_boundaries=True))
    text = ' $$ '.join(sbd.sentences_from_text(text, realign_boundaries=True))
    text = text.replace('[','').replace(']','')
    text = ''.join(mystem.lemmatize(text.strip()))
    text = re.sub('[^а-яА-Я\-$ ]','', text)
    #text = re.sub('[0-9—«»’§°¼½×˃<…“„]+','', text)
    text = re.sub('[-]{2,}','', text)
    text = text.split(' $$ ')
    for i in text:
        i2 = i.translate(i.maketrans('','',string.punctuation))
        i2 = re.sub('[\n\s]+',' ', i2)
        lid = langid.classify(i2)[0]
        if lid not in ['fr','la','el'] and len(i2)>25:
            f.write(i2.strip()+'\n')
            pass
    return lng

In [48]:
def universal_preprocessing(path, 
                            tags_for_deletion, 
                            tags_to_strip, 
                            filename, printing=True):
    tags_for_deletion = tags_for_deletion
    tags_to_strip = tags_to_strip
    mystem = Mystem()
    start_gen = datetime.now()
    with open (filename, 'w', encoding='utf-8') as f:
        for root, dirs, files in os.walk (path):
                for fl in files :
                    start = datetime.now() 
                    lng = preprocessing(get_one_writing(root+'\\'+fl), f)
                    if printing:
                        print ('{}\t{}\t{}\t{}'.format(lng, os.path.getsize(root+'\\'+fl)//1024, 
                                              datetime.now()-start,
                                              fl))
    print ('\n\ntotal time:\t\t {}'.format(datetime.now()-start_gen))
    

### Произведения

** Плохие теги: **

- note
- add
- sic
- orig
- head
- del
- figure
- table


** Ненужные теги: **

- pb
- hi
- corr
- reg
- choice
- div
- p

In [10]:
tags_for_deletion  = ['.//tei:note','.//tei:add','.//tei:sic','.//tei:orig',
                      './/tei:head','.//tei:del','.//tei:figure','.//tei:table']

tags_to_strip = ['{http://www.tei-c.org/ns/1.0}pb', '{http://www.tei-c.org/ns/1.0}hi', '{http://www.tei-c.org/ns/1.0}corr',
                '{http://www.tei-c.org/ns/1.0}reg', '{http://www.tei-c.org/ns/1.0}choice', '{http://www.tei-c.org/ns/1.0}p',
                '{http://www.tei-c.org/ns/1.0}div', '{http://www.tei-c.org/ns/1.0}lb']

НЕ ЗАПУСКАТЬ!!!

In [ ]:
path = '.\\data\\Final\\'
universal_preprocessing(path, tags_for_deletion, tags_to_strip, 'writings.txt')

In [73]:
s = 'Je conçois que cest ainsi que tu las voulu'
langid.classify(s)[0]

'pt'

### Письма

opener
note
pb
closer

In [43]:
tags_for_deletion  = ['.//tei:note','.//tei:add','.//tei:sic','.//tei:orig',
                      './/tei:head','.//tei:del','.//tei:figure','.//tei:table',
                      './/tei:opener', './/tei:closer']

tags_to_strip = ['{http://www.tei-c.org/ns/1.0}pb', '{http://www.tei-c.org/ns/1.0}hi', '{http://www.tei-c.org/ns/1.0}corr',
                '{http://www.tei-c.org/ns/1.0}reg', '{http://www.tei-c.org/ns/1.0}choice', '{http://www.tei-c.org/ns/1.0}p',
                '{http://www.tei-c.org/ns/1.0}div', '{http://www.tei-c.org/ns/1.0}lb','{http://www.tei-c.org/ns/1.0}body']

In [25]:
with open (".\\data\\LETTERS\\Volume_59\\letter101.xml", 'r', encoding='utf-8') as f:
    s = f.read()
    s = re.sub('<note.*?>', '<note>', s)
with open ('1.xml','w', encoding='utf-8') as f:
    f.write(s)
#print (s)
root = etree.parse('1.xml')

In [55]:
path = '.\\data\\LETTERS\\'
for root, dirs, files in os.walk (path):
    for fl in files :
        with open (root+'\\'+fl, 'r', encoding='utf-8') as f:
            s = f.read()
            s = re.sub('<note.*?>', '<note>', s)
            s = re.sub('xml:id=".*?"','',s)
        with open (root+'\\'+fl,'w', encoding='utf-8') as f:
            f.write(s)

In [56]:
path = '.\\data\\LETTERS\\'
universal_preprocessing(path, tags_for_deletion, tags_to_strip, 'letters.txt', printing=False)



total time:		 4:39:45.492822


### Дневники

opener
note



div
p

In [21]:
tags_for_deletion  = ['.//tei:note','.//tei:add','.//tei:sic','.//tei:orig',
                      './/tei:head','.//tei:del','.//tei:figure','.//tei:table', './/tei:opener']

tags_to_strip = ['{http://www.tei-c.org/ns/1.0}pb', '{http://www.tei-c.org/ns/1.0}hi', '{http://www.tei-c.org/ns/1.0}corr',
                '{http://www.tei-c.org/ns/1.0}reg', '{http://www.tei-c.org/ns/1.0}choice', '{http://www.tei-c.org/ns/1.0}p',
                '{http://www.tei-c.org/ns/1.0}div', '{http://www.tei-c.org/ns/1.0}lb','{http://www.tei-c.org/ns/1.0}body']

In [22]:
path = '.\\data\\diaries\\'
universal_preprocessing(path, tags_for_deletion, tags_to_strip, 'diaries.txt')

845	981	0:00:10.897919	46.xml
524	698	0:00:04.896199	47.xml
2675	268	0:00:17.522655	48.xml
4700	425	0:00:28.088938	49.xml
6204	778	0:00:38.901313	50.xml
3504	441	0:00:23.632458	51.xml
3829	612	0:00:27.714655	52.xml
4721	897	0:00:33.016014	53.xml
1583	752	0:00:13.058729	54.xml
2210	1117	0:00:20.002205	55.xml
2576	639	0:00:20.198545	56.xml
2834	733	0:00:21.618889	57.xml
2452	568	0:00:17.266406	58.xml


total time:		 {} 0:04:36.820934


In [15]:
filename = ".\data\diaries\\46.xml"
root = etree.parse(filename)#.getroot()
for i in tags_for_deletion:
    for j in root.findall(i, tei_ns):
        parent = j.find("..")
        parent.remove(j)
for i in tags_to_strip:
    etree.strip_tags(root, i)
root.find(".//tei:text", tei_ns).text.strip()
#etree.tostring(root)

"Казань]. 17 Мар. Вотъ уже шесть дней, какъ я поступилъ въ клинику, и вотъ шесть дней, какъ я почти доволенъ собою. Les petites causes produisent de grands effets.Гаонарею, понимается, отъ того, отъ чего она обыкновенно получается; и это пустое обстоятельство дало мнѣ толчокъ, отъ котораго я сталъ на ту ступень, на которой я уже давно поставилъ ногу; но никакъ не могъ перевалить туловище (отъ того, должно быть, что не обдумавши поставилъ лѣвую ногу вмѣсто правой). Здѣсь я совершенно одинъ, мнѣ никто не мѣшаетъ, здѣсь у меня нѣтъ услуги, мнѣ никто не помогаетъ — слѣдовательно на разсудокъ и память ничто постороннее не имѣетъ вліянія, и дѣятельность моя необходимо должна развиваться. Главная же польза состоитъ въ томъ, что я ясно усмотрѣлъ, что безпорядочная жизнь, которую большая часть свѣтскихъ людей принимаютъ зa слѣдствіе молодости, есть ничто иное, какъ слѣдствіе ранняго разврата души.очки,3 4которые показывали ему все въ превратномъ видѣ, и какъ уяснится взглядъ его на вѣщи, такъ ч

## Обработка текста

# Try

In [18]:
import gensim

C:\Users\Glaz\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [88]:
sentences = gensim.models.word2vec.LineSentence('writings.txt')

In [82]:
#sentences = [['я', 'иду'], ['я', 'иду']]

In [94]:
#start = datetime.now() 
%time
model = gensim.models.Word2Vec(sentences, min_count=2, sorted_vocab=1)
#print(datetime.now()-start)

Wall time: 0 ns


In [95]:
print(model)

Word2Vec(vocab=29551, size=100, alpha=0.025)


In [ ]:
print(model.wv.vocab.keys())